## CSCI59 - Final Project
### FHIR Data for Pharmacy
#### Erin Rebholz, Spring 2024

### **Imports**

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
#!pip install fhiry

**fhiry extracts multiple patient data files into a single data frame**

#https://github.com/dermatologist/fhiry

In [3]:
import fhiry.parallel as fp
df_full = fp.process('data_4r/')
print(df_full.info())

<class 'pandas.core.frame.DataFrame'>
Index: 409973 entries, 0 to 983
Columns: 159 entries, fullUrl to resource.organization.displaydisplay
dtypes: float64(11), object(148)
memory usage: 500.5+ MB
None


In [4]:
#Export to excel to review quality
#df_full.iloc[:5000,:].to_csv('tester.csv')

In [4]:
#Replce the dots with dashes for better compatibility
df_full.columns = df_full.columns.str.replace('.','_')

In [5]:
len(df_full)

409973

**Patient Data**

In [6]:
def patient_data_parser(df_full):
    #Create patient data data frame
    patient_data = df_full[df_full['resource_resourceType'] == 'Patient'].reset_index()
    pat_cols = ['resource_resourceType', 'id', 'resource_name', 'resource_telecom',
           'resource_gender', 'resource_birthDate', 'resource_deceasedDateTime',
           'resource_address','resource_subject_reference']
    patient_df = patient_data.loc[:,pat_cols]
    
    #Normalize to decesed date (strip time)
    patient_df['resource_deceasedDateTime_norm'] = ['1000-01-01' if type(ii)!=(str) else ii[:10] for ii in  patient_df.resource_deceasedDateTime]
    #Parse Resource Address into Sub-fields
    patient_df['resource_address_line'] = [pd.json_normalize(x).loc[0,'line'][0] for x in patient_df['resource_address']] 
    patient_df['resource_address_city'] = [pd.json_normalize(x).loc[0,'city'] for x in patient_df['resource_address']] 
    patient_df['resource_address_state'] = [pd.json_normalize(x).loc[0,'state'] for x in patient_df['resource_address']] 
    ### Note that not all entries had 'postal code' so we need to assign "" in the second line
    patient_df['resource_address_postalCode'] = [pd.json_normalize(x).iloc[0,4] for x in patient_df['resource_address']] 
    patient_df['resource_address_postalCode'] = ["" if x == 'US' else x for x in patient_df['resource_address_postalCode']]
    patient_df['resource_address_country'] = [pd.json_normalize(x).loc[0,'country'] for x in patient_df['resource_address']] 

    #Parse Resource Name into sub-fields
    patient_df['resource_name_given'] = [pd.json_normalize(x).iloc[0,1] for x in patient_df['resource_name']]
    patient_df['resource_name_family'] = [pd.json_normalize(x).iloc[0,2][0] for x in patient_df['resource_name']] #Enclosed in brackets
    #Extract phone info from telecom
    patient_df['resource_telecom_phone'] = [pd.json_normalize(x).iloc[0,1] for x in patient_df['resource_telecom']] 

    
    #Reorder columns and get rid of unnecessary columns
    cols = [ 'id', 'resource_name_given','resource_name_family',
           'resource_gender', 'resource_birthDate', 'resource_deceasedDateTime',
            'resource_address_line', 'resource_address_city',
           'resource_address_state', 'resource_address_postalCode',
           'resource_address_country',  'resource_telecom_phone']
    
    patient_df = patient_df.loc[:,[ 'id', 'resource_name_given','resource_name_family',
           'resource_gender', 'resource_birthDate', 'resource_deceasedDateTime_norm',
            'resource_address_line', 'resource_address_city',
           'resource_address_state', 'resource_address_postalCode',
           'resource_address_country',  'resource_telecom_phone']]
    
    #rename columns to friendly names
    patient_df.columns = [ 'patient_id', 'given_name','family_name',
           'gender', 'birth_date', 'deceased_date',
            'street_address', 'city',
           'state', 'postal_code',
           'country',  'phone']
    
    return patient_df

In [7]:
patient_df = patient_data_parser(df_full)

In [8]:
#check Encounter_ID is unique
len(patient_df), len(patient_df.patient_id.unique())

(555, 555)

In [9]:
patient_df.to_csv('tables/patient_table.csv', index = False)

In [10]:
patient_df.head()

,patient_id,given_name,family_name,gender,birth_date,deceased_date,street_address,city,state,postal_code,country,phone
0,b0a06ead-cc42-aa48-dad6-841d4aa679fa,Harris789,Abdul218,male,1952-12-05,2002-02-01,572 Keebler Esplanade,Boston,MA,02134,US,555-443-1041
1,ccfc4db2-2026-7adb-3db0-33f3828140bb,Schmidt332,Abe604,male,1976-06-06,1000-01-01,768 Mertz Camp,Peabody,MA,01940,US,555-345-5342
2,31a2e8ec-69fc-8a71-3ab6-36cbdd508713,DuBuque211,Adelaida985,female,1917-05-15,2017-02-18,848 Casper Way,Quincy,MA,02169,US,555-925-4660
3,71a8b156-760b-df6b-859e-eefc7932a526,Hintz995,Adriana394,female,2014-11-15,1000-01-01,455 Bayer Rapid Suite 75,Boston,MA,02120,US,555-237-5240
4,76b289fd-e825-734c-8446-316f59643593,Gerhold939,Agueda283,female,1985-03-03,1000-01-01,733 Bednar Grove Apt 36,Boston,MA,02115,US,555-831-3391


**Observation**

In [11]:
def obs_data_parser(df_full):

    #Create observation data data frame
    obs_data = df_full[df_full['resource_resourceType'] == 'Observation'].reset_index()
    
    #Select columns for exploration
    obs_cols = [1,2,3,18,23,29,32,35,36,47,65,72,73,75]
    obs_df = obs_data.iloc[:, obs_cols]
    
    #Normalize date
    obs_df['effective_date'] = [ii if type(ii)!=(str) else ii[:10] for ii in  obs_df.resource_effectiveDateTime]
    
    #Truncate Encounter and Patient Reference Keys to Link Back to Other Table's Formats
    obs_df['encounter_id'] = [ii[9:] for ii in obs_df.resource_encounter_reference]
    obs_df['patient_id'] = [ii[9:] for ii in obs_df.resource_subject_reference]
    
    #Observation Code (strip unneeded data and normalize json dictionary stuff
    obs_df['obs_code'] = [pd.json_normalize(pd.json_normalize(x).iloc[0,0]).iloc[0,1] for x in obs_df['resource_category']]  # this is slow, refactor?
    
    
    #Reshuffle columns and remove not needed columns)
    obs_cols = ['id', 'encounter_id', 'patient_id', 'effective_date',
           'obs_code','resource_code_text',
           'resource_valueQuantity_value', 'resource_valueQuantity_unit',
           'resource_valueQuantity_code']
    
    obs_df = obs_df.loc[:,obs_cols]
    
    #rename columns to friendly names
    obs_df.columns = ['obs_id', 'encounter_id', 'patient_id', 'effective_date', 'obs_code',
                      'code_text', 'quantity_value',
                      'quantity_unit', 'quantity_code']

    return obs_df

In [12]:
obs_df = obs_data_parser(df_full)

C:\Users\erinr\AppData\Local\Temp\ipykernel_14224\2799416663.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_df['effective_date'] = [ii if type(ii)!=(str) else ii[:10] for ii in  obs_df.resource_effectiveDateTime]
C:\Users\erinr\AppData\Local\Temp\ipykernel_14224\2799416663.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_df['encounter_id'] = [ii[9:] for ii in obs_df.resource_encounter_reference]
C:\Users\erinr\AppData\Local\Temp\ipykernel_14224\2799416663.py:15: SettingWithCopyWarning: 
A 

In [13]:
#check Observation_ID is unique
len(obs_df), len(obs_df.obs_id.unique())

(131703, 131703)

In [14]:
obs_df.to_csv('tables/obs_table.csv', index= False)

In [15]:
obs_df.head()

,obs_id,encounter_id,patient_id,effective_date,obs_code,code_text,quantity_value,quantity_unit,quantity_code
0,33648be3-5286-a324-312c-c9cf3cfedac8,d331b9e7-cc03-6a74-ecac-64fedddb57a9,b0a06ead-cc42-aa48-dad6-841d4aa679fa,1992-04-10,vital-signs,Body Height,186.00,cm,cm
1,a5cd0f82-45c2-1f1c-b5cb-24d317cfa74b,d331b9e7-cc03-6a74-ecac-64fedddb57a9,b0a06ead-cc42-aa48-dad6-841d4aa679fa,1992-04-10,vital-signs,Pain severity - 0-10 verbal numeric rating [Sc...,1.00,{score},{score}
2,75a7476a-de6c-ce31-5b47-e5e72e7c4211,d331b9e7-cc03-6a74-ecac-64fedddb57a9,b0a06ead-cc42-aa48-dad6-841d4aa679fa,1992-04-10,vital-signs,Body Weight,102.80,kg,kg
3,26945d93-ed09-2c27-55d2-698657b29dcf,d331b9e7-cc03-6a74-ecac-64fedddb57a9,b0a06ead-cc42-aa48-dad6-841d4aa679fa,1992-04-10,vital-signs,Body Mass Index,29.71,kg/m2,kg/m2
4,fa2b75e0-a7aa-5fdb-34d8-c88c3addc246,d331b9e7-cc03-6a74-ecac-64fedddb57a9,b0a06ead-cc42-aa48-dad6-841d4aa679fa,1992-04-10,vital-signs,Blood Pressure,NaN,NaN,NaN


**Encounter**

In [16]:
def encounter_data_parser(df_full):

    enc_data = df_full[df_full['resource_resourceType'] == 'Encounter'].reset_index()
    
    #Select columns for exploration
    enc_cols = ['id','resource_class_code','resource_type','resource_subject_reference',
                'resource_participant','resource_period_start','resource_period_end','resource_location']
    
    enc_df = enc_data.loc[:, enc_cols]
    
    #normalize Date times to dates
    enc_df['start_date'] = [ii if type(ii)!=(str) else ii[:10] for ii in  enc_df.resource_period_start]
    enc_df['end_date'] = [ii if type(ii)!=(str) else ii[:10] for ii in  enc_df.resource_period_end]
    
    #visit info
    enc_df['snomed_code'] = [pd.json_normalize(pd.json_normalize(x).loc[0,'coding'][0])['code'][0] for x in enc_df['resource_type']]
    enc_df['snomed_text'] = [pd.json_normalize(pd.json_normalize(x).loc[0,'coding'][0])['display'][0] for x in enc_df['resource_type']]
    
    #provider info
    enc_df['provider_NPI'] = [pd.json_normalize(x)['individual.reference'][0][-10:]  for x in enc_df.resource_participant]
    enc_df['provider_name'] = [pd.json_normalize(x)['individual.display'][0]  for x in enc_df.resource_participant]
    
    #facility info
    enc_df['facility_name'] = [pd.json_normalize(x)['location.display'][0]  for x in enc_df.resource_location]
    
    #patient id
    enc_df['patient_id'] = [ii[9:] for ii in enc_df.resource_subject_reference]
    
    
    #shuffle and remove column names
    enc_df_cols = ['id', 'patient_id', 'start_date', 'end_date','snomed_code', 'snomed_text', 'provider_NPI',
                   'provider_name', 'facility_name','resource_class_code']
    
    enc_df = enc_df.loc[:,enc_df_cols]
    
    #rename columns to friendly names
    enc_df.columns = ['encounter_id', 'patient_id', 'start_date', 'end_date', 
           'snomed_code', 'snomed_text', 'provider_NPI', 'provider_name',
           'facility_name','facility_code']
    return enc_df

In [17]:
enc_df = encounter_data_parser(df_full)

In [18]:
enc_df.to_csv('tables/enc_table.csv', index = False)

In [19]:
#check Encounter_ID is unique
len(enc_df), len(enc_df.encounter_id.unique())

(27812, 27812)

In [38]:
enc_df.head(3)

,encounter_id,patient_id,start_date,end_date,snomed_code,snomed_text,provider_NPI,provider_name,facility_name,facility_code
0,a889805b-20ca-0ba6-5b5c-e617ce99b0cc,b0a06ead-cc42-aa48-dad6-841d4aa679fa,1954-04-03,1954-04-03,185347001,Encounter for problem,9999999989,Dr. Lesley194 Fisher429,MOUNT AUBURN HOSPITAL,AMB
1,b1ca5d4d-e3b5-e65b-e900-3844ae30f421,b0a06ead-cc42-aa48-dad6-841d4aa679fa,1954-04-20,1954-04-20,185347001,Encounter for problem,9999999989,Dr. Lesley194 Fisher429,MOUNT AUBURN HOSPITAL,AMB
2,853a4e65-da0d-d531-eb6e-458a70bcf552,b0a06ead-cc42-aa48-dad6-841d4aa679fa,1959-06-03,1959-06-03,185345009,Encounter for symptom,9999999989,Dr. Lesley194 Fisher429,MOUNT AUBURN HOSPITAL,AMB


**Medication History**

In [123]:
def medication_data_parser(df_full):
    med_data = df_full[df_full['resource_resourceType'] == 'MedicationRequest'].reset_index()
        
    #Select columns for exploration
    med_cols_raw = ['id','resource_status','resource_subject_reference','resource_encounter_reference',
                    'resource_intent','resource_medicationCodeableConcept_text', 'resource_authoredOn',
                    'resource_requester_reference','resource_prescription_reference','patientId',
                    'resource_medicationCodeableConcept_codingcodes','resource_dosageInstruction']
    
    med_df = med_data.loc[:, med_cols_raw]
    
    #normalize Date times to dates
    med_df['request_date'] = [ ii[:10] for ii in  med_df.resource_authoredOn]
    
    #Truncate Encounter and Patient Reference Keys to Link Back to Other Table's Formats
    med_df['encounter_id'] = [ii[9:] for ii in med_df.resource_encounter_reference]
    med_df['patient_id'] = [ii[9:] for ii in med_df.resource_subject_reference]
    
    #Rx_Norm
    med_df['rxnorm_code'] = med_df.resource_medicationCodeableConcept_codingcodes.astype('str').str.strip('[').str.strip(']').str.strip("'")
    med_df['rxnorm_text'] =  med_df.resource_medicationCodeableConcept_text.replace(',',' ')
    
    #NPI
    med_df['provider_id'] = [x[-10:] for x in med_df.resource_requester_reference]
    
    
    ###Dosing Instructions
    text = []
    snomed_code = []
    snomed_text = []
    repeat_freq = []
    repeat_period = []
    repeat_period_unit = []
    as_needed = []
    
    for ii in range(len(med_df.resource_dosageInstruction)):
        try:
            helper = med_df.resource_dosageInstruction[ii][0]['text']
            text.append(helper)
        except: 
            text.append("")
    
        try: 
            helper = med_df.resource_dosageInstruction[ii][0]['timing']
            helper1 = helper['repeat']['frequency']
            helper2 = helper['repeat']['period']
            helper3 = helper['repeat']['periodUnit']
            
            repeat_freq.append(helper1)
            repeat_period.append(helper2)
            repeat_period_unit.append(helper3)
        except: 
            repeat_freq.append("")
            repeat_period.append("")
            repeat_period_unit.append("")
        
        try:
            helper = med_df.resource_dosageInstruction[100][0]['asNeededBoolean']
            as_needed.append(helper)
        except: 
            as_needed.append("")
    
        try:
            helper = med_df.resource_dosageInstruction[ii][0]['additionalInstruction'][0]['coding'][0]['code'] #snomed code
            snomed_code.append(helper)
            helper = med_df.resource_dosageInstruction[ii][0]['additionalInstruction'][0]['coding'][0]['display'] #snomed text
            snomed_text.append(helper)
        except:
            snomed_code.append('')
            snomed_text.append('')
    
    med_df['rx_text'] = text
    med_df['snomed_code'] = snomed_code
    med_df['snomed_text'] = snomed_text
    med_df['repeat_freq'] = repeat_freq
    med_df['repeat_period'] = repeat_period
    med_df['repeat_period_unit'] = repeat_period_unit
    med_df['as_needed'] = as_needed
       
    med_df['rx_text'] =  med_df.rx_text.replace(',',' ')
    
    #Reorder columns and omit unnecessary values:
    med_cols = ['id', 'resource_status', 'patient_id', 'provider_id', 'encounter_id',
                'request_date', 'rxnorm_code', 'rxnorm_text','rx_text',
                'snomed_code', 'snomed_text', 'repeat_freq', 'repeat_period','repeat_period_unit', 'as_needed']
    
    
    med_df = med_df.loc[:,med_cols]

    med_df.columns = ['rx_id', 'resource_status', 'patient_id', 'provider_id', 'encounter_id',
                'request_date', 'rxnorm_code', 'rxnorm_text','rx_text',
                'snomed_code', 'snomed_text', 'repeat_freq', 'repeat_period','repeat_period_unit', 'as_needed']
    
    return med_df

In [124]:
med_df = medication_data_parser(df_full)

In [125]:
med_df.to_csv('tables/med_history.csv', index = False, encoding='utf-8',quotechar = '"')

In [25]:
#check Rx_ID is unique
len(med_df), len(med_df.rx_id.unique())

(24256, 24256)

In [26]:
med_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24256 entries, 0 to 24255
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   rx_id               24256 non-null  object
 1   resource_status     24256 non-null  object
 2   patient_id          24256 non-null  object
 3   provider_id         24256 non-null  object
 4   encounter_id        24256 non-null  object
 5   request_date        24256 non-null  object
 6   rxnorm_code         24256 non-null  object
 7   rxnorm_text         23902 non-null  object
 8   rx_text             24256 non-null  object
 9   snomed_code         24256 non-null  object
 10  snomed_text         24256 non-null  object
 11  repeat_freq         24256 non-null  object
 12  repeat_period       24256 non-null  object
 13  repeat_period_unit  24256 non-null  object
 14  as_needed           24256 non-null  bool  
dtypes: bool(1), object(14)
memory usage: 2.6+ MB


In [126]:
med_df['rxnorm_text'] =  med_df.rxnorm_text.str.strip(',')
#med_df['rxnorm_code'] = med_df.resource_medicationCodeableConcept_codingcodes.str.strip('[').str.strip(']')

In [127]:
med_df.head()

,rx_id,resource_status,patient_id,provider_id,encounter_id,request_date,rxnorm_code,rxnorm_text,rx_text,snomed_code,snomed_text,repeat_freq,repeat_period,repeat_period_unit,as_needed
0,c0e50da8-7aed-f2db-f893-efe740504150,stopped,b0a06ead-cc42-aa48-dad6-841d4aa679fa,9999999989,d331b9e7-cc03-6a74-ecac-64fedddb57a9,1992-04-10,312961,Simvastatin 20 MG Oral Tablet,,,,,,,False
1,28fe47d9-82cf-c35d-468d-39ecd61c3b9c,stopped,b0a06ead-cc42-aa48-dad6-841d4aa679fa,9999999989,d331b9e7-cc03-6a74-ecac-64fedddb57a9,1992-04-10,705129,Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray,,,,,,,False
2,4d93f56b-3db5-e0a6-a4c7-f65cf2b8538d,stopped,b0a06ead-cc42-aa48-dad6-841d4aa679fa,9999999989,d331b9e7-cc03-6a74-ecac-64fedddb57a9,1992-04-10,705129,Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray,,,,,,,False
3,9f5c47fb-b639-e236-ea1a-b6bc2fcab325,stopped,b0a06ead-cc42-aa48-dad6-841d4aa679fa,9999999989,d331b9e7-cc03-6a74-ecac-64fedddb57a9,1992-04-10,312961,Simvastatin 20 MG Oral Tablet,,,,,,,False
4,e7a52e7d-d8ec-5824-b4a6-bb936b2f299a,stopped,b0a06ead-cc42-aa48-dad6-841d4aa679fa,9999968019,f2f337b4-33bb-1fe1-4f96-8102f9a8b75a,1992-04-24,705129,Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray,,,,,,,False


**Allergy**

In [29]:
def allergy_data_parser(df_full):

    #Create allergy data frame
    al_data = df_full[df_full['resource_resourceType'] == 'AllergyIntolerance'].reset_index()
    
    #Select columns for exploration
    al_cols_raw = ['resource_resourceType', 'id', 'request_url', 'resource_type',
           'resource_category', 'resource_patient_reference',
           'resource_criticality', 'resource_code_text', 'resource_recordedDate',
           'resource_reaction']
        
    al_df = al_data.loc[:, al_cols_raw]
    
    #normalize Date times to dates
    al_df['recorded_date'] = [ ii[:10] for ii in  al_df.resource_recordedDate]
        
    #Truncate Patient Reference Keys to Link Back to Other Table's Formats
    al_df['patient_id'] = [ii[9:] for ii in al_df.resource_patient_reference]

    al_df['resource_category'] = al_df.resource_category.astype('str').str.strip('[').str.strip(']').str.strip("'")
    
    ## Get allergy symptoms from nested JSON column
    text = []
    
    for ii in range(len(al_df.resource_reaction)):
        try:
            helper = ""
            for i in range(len(al_df.iloc[ii,9])):
                helper = helper + (f"{i+1}: {al_df.iloc[ii,9][i]['manifestation'][0]['coding'][0]['display']} ")
            text.append(helper)
        except: 
            text.append("")
    
    al_df['reaction'] = text
    
    #Reorder columns and omit unnecessary values:
    al_cols = ['id', 'patient_id', 'recorded_date','resource_category', 'resource_code_text', 'reaction', 'resource_criticality']
        
    al_df = al_df.loc[:,al_cols]
    
    al_df.columns = ['allergy_id', 'patient_id', 'recorded_date','allergy_category', 'allergy_details', 'reaction', 'allergy_criticality']
    
    return al_df

In [30]:
al_df = allergy_data_parser(df_full)

In [31]:
al_df.to_csv('tables/allergy_table.csv',index = False, encoding='utf-8')

In [32]:
#check Allergy_ID is unique
len(al_df), len(al_df.allergy_id.unique())

(499, 499)

In [37]:
al_df.allergy_criticality.value_counts()

allergy_criticality
low    499
Name: count, dtype: int64

In [33]:
al_df.head()

,allergy_id,patient_id,recorded_date,allergy_category,allergy_details,reaction,allergy_criticality
9,7f020d47-51c8-740e-5233-1a6ef6484ddb,0a06ead-cc42-aa48-dad6-841d4aa679fa,1954-04-20,food,Eggs (edible) (substance),,low
168,738f95d5-e2c3-c8e8-bdd9-61eba55a4ad6,2fb7efc-5cfd-f8d3-927b-42f8ee099531,2014-12-14,food,Fish (substance),1: Dyspnea (finding) 2: Finding of vomiting (f...,low
9,5692c338-6acd-0a43-7a77-ca3e3cfbf370,46a1435-2455-914f-c287-7b88052d05db,1983-05-11,environment,Mold (organism),1: Allergic skin rash,low
10,44c35163-e3a9-05f1-aeda-4bb52e700a84,46a1435-2455-914f-c287-7b88052d05db,1983-05-11,environment,House dust mite (organism),,low
11,2efa474b-ffe8-6013-042c-8043c36f55a1,46a1435-2455-914f-c287-7b88052d05db,1983-05-11,environment,Animal dander (substance),1: Eruption of skin (disorder) 2: Wheal (findi...,low


### **Patient Data Loading for SQL**

**Search for patients with insulin perscriptions**

In [130]:
#Patients with active insulin prescriptons
diabetes_patients = med_df[(med_df.rxnorm_text.str.contains('insulin')) &(med_df.resource_status == 'active')].patient_id.unique()
diabetes_patients

array(['9e84e569-7adc-ff42-ccdb-9fe9c23842a6',
       '060e72d3-912e-55cd-0c92-5faa6cb7a6db',
       '8c6ae452-5f8c-9ff6-006d-c6c860acf5cd',
       'be82309d-1a8f-df82-4cd6-5f03e1060e8e',
       '3dfce025-b696-711a-8c2d-6903736e2882',
       '55c5b8d3-99d0-58ad-8444-e42bb81bd5c7',
       'ed803173-5e55-aecd-5055-591164ee3005',
       'ef167059-cef0-12c4-49db-993ca3a20c01',
       'fa2d28b1-de84-ac96-ee04-a20dc54832c1',
       'e23c7a6e-6ef2-3932-1361-b73b4c8a3961',
       'd57e867e-4896-9e07-f502-5f4f881bd80e'], dtype=object)

In [144]:
patient_sample =  patient_df[patient_df.patient_id.isin(diabetes_patients)]
len(patient_sample)

11

In [131]:
#List of past medications
med_sample = med_df[med_df.patient_id.isin(diabetes_patients)]
active_med_sample = med_df[(med_df.patient_id.isin(diabetes_patients)) &(med_df.resource_status == 'active')]

len(med_sample),len(active_med_sample)

(6935, 66)

In [136]:
code_text_samples = ['Body Weight', 'Body Mass Index','Blood Pressure','Hemoglobin A1c/Hemoglobin.total in Blood','Total Cholesterol',
                     'Low Density Lipoprotein Cholesterol','High Density Lipoprotein Cholesterol','Creatinine','Microalbumin Creatinine Ratio',
                    'Creatinine [Mass/volume] in Serum or Plasma']

obs_sample = obs_df[(obs_df.patient_id.isin(diabetes_patients))&(obs_df.code_text.isin(code_text_samples))]
len(obs_sample)

5035

In [137]:
enc_sample = enc_df[enc_df.patient_id.isin(diabetes_patients)]
len(enc_sample)

3996

In [138]:
al_sample = al_df[al_df.patient_id.isin(diabetes_patients)]
len(al_sample)

5

**Create a function to generate the sql text to insert patient**

In [195]:
sql_list = []

encounter = True
med_history = True
observation = True
allergy = True

for i in range(len(diabetes_patients)):
#for i in range(1):  #limited # of patients
    #Generate new patient text
    small_df = patient_sample[patient_sample.patient_id == diabetes_patients[i]]

    for j in range(patient_sample.shape[1]):
        if j == 0:
            text = '"'+small_df.iloc[0,j]+'"'
        else:   
            text = text+' ,"'+small_df.iloc[0,j]+'"'

    helper = f'INSERT INTO patient(patient_id, given_name, family_name, gender, birth_date, deceased_date,street_address,city,state,postal_code, country, phone) VALUES ({text});'
    
    sql_list.append(helper)

    #Generate encounter data set
    if encounter == True:
        small_df = enc_sample[enc_sample.patient_id == diabetes_patients[i]]
        for k in range(small_df.shape[0]):
            for j in range(small_df.shape[1]):
                if j == 0:
                    text = '"'+small_df.iloc[k,j]+'"'
                else:   
                    text = text+' ,"'+small_df.iloc[k,j]+'"'
            
            helper = f'INSERT INTO encounter(encounter_id,patient_id, start_date, end_date, snomed_code, snomed_text, provider_NPI, provider_name,facility_name, facility_code) VALUES ({text});'
            sql_list.append(helper)    
    else:
        pass
        
    #Generate med history data set
    if med_history == True:
        small_df = med_sample[med_sample.patient_id == diabetes_patients[i]]
        for k in range(small_df.shape[0]):
            for j in range(small_df.shape[1]):
                if j == 0:
                    text = '"'+small_df.iloc[k,j]+'"'
                else:   
                    text = text+' ,"'+str(small_df.iloc[k,j])+'"'
            
            helper = f'INSERT INTO med_history(rx_id, resource_status,	patient_id, provider_id, encounter_id, request_date, rxnorm_code, rxnorm_text, rx_text, snomed_code, snomed_text,repeat_freq,repeat_period,repeat_period_unit, as_needed) VALUES ({text});'
            sql_list.append(helper)
    else:
        pass
        
    #Generate observation data set
    if observation == True:
        small_df = obs_sample[obs_sample.patient_id == diabetes_patients[i]]
        for k in range(small_df.shape[0]):
            for j in range(small_df.shape[1]):
                if j == 0:
                    text = '"'+small_df.iloc[k,j]+'"'
                else:   
                    text = text+' ,"'+str(small_df.iloc[k,j])+'"'
                
            helper = f'INSERT INTO observation (obs_id, encounter_id, patient_id, effective_date, obs_code, code_text, quantity_value, quantity_unit, quantity_code) VALUES ({text});'
            sql_list.append(helper)
    else:
        pass

    #Generate allergy data set
    if allergy == True:
        small_df = al_sample[al_sample.patient_id == diabetes_patients[i]]
        for k in range(small_df.shape[0]):
            for j in range(small_df.shape[1]):
                if j == 0:
                    text = '"'+small_df.iloc[k,j]+'"'
                else:   
                    text = text+' ,"'+str(small_df.iloc[k,j])+'"'
                
            helper = f'INSERT INTO allergy (allergy_id, patient_id, recorded_date, allergy_category, allergy_details, reaction, allergy_criticality) VALUES ({text});'
            sql_list.append(helper)

    else:
        pass



In [196]:
sql_list

['INSERT INTO patient(patient_id, given_name, family_name, gender, birth_date, deceased_date,street_address,city,state,postal_code, country, phone) VALUES ("9e84e569-7adc-ff42-ccdb-9fe9c23842a6" ,"Hilll811" ,"Armando772" ,"male" ,"1945-11-04" ,"1000-01-01" ,"397 Rodriguez Promenade Suite 93" ,"Lynn" ,"MA" ,"01902" ,"US" ,"555-296-4764");',
 'INSERT INTO encounter(encounter_id,patient_id, start_date, end_date, snomed_code, snomed_text, provider_NPI, provider_name,facility_name, facility_code) VALUES ("f42b5a45-0e30-c707-4618-14ba9e107390" ,"9e84e569-7adc-ff42-ccdb-9fe9c23842a6" ,"1963-12-29" ,"1963-12-29" ,"162673000" ,"General examination of patient (procedure)" ,"9999954489" ,"Dr. Marian936 Wiza601" ,"PCP112182" ,"AMB");',
 'INSERT INTO encounter(encounter_id,patient_id, start_date, end_date, snomed_code, snomed_text, provider_NPI, provider_name,facility_name, facility_code) VALUES ("b85d8e8a-67aa-9d12-1038-76b34f73c330" ,"9e84e569-7adc-ff42-ccdb-9fe9c23842a6" ,"1964-01-28" ,"1964-01-

In [198]:
file = open('patient.txt','w')
for item in sql_list:
	file.write(item+"\n")
file.close()